In [1]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler
from custom_datasets import CustomDataset
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import warnings
from sklearn.model_selection import KFold

/Users/seungyoo/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
warnings.filterwarnings(action='ignore')
from sklearn.metrics import make_scorer
#define your own mse and set greater_is_better=False
mse = make_scorer(mean_squared_error,greater_is_better=False)

In [3]:
df_train = pd.read_csv('../datasets/kitti_train.csv')
df_valid = pd.read_csv('../datasets/kitti_valid.csv')
df_test = pd.read_csv('../datasets/kitti_test.csv')

In [4]:
train = df_train.drop(['filename', 'class', 'weather'], axis=1)
valid = df_valid.drop(['filename', 'class', 'weather'], axis=1)
test = df_test.drop(['filename', 'class', 'weather'], axis=1)

In [5]:
X_train = train.loc[:, train.columns != 'zloc']
y_train = train.loc[:, train.columns == 'zloc']

X_valid = valid.loc[:, valid.columns != 'zloc']
y_valid = valid.loc[:, valid.columns == 'zloc']

X_test = test.loc[:, test.columns != 'zloc']
y_test = test.loc[:, test.columns == 'zloc']

In [6]:
X_train.head()

,xmin,ymin,xmax,ymax,angle,depth_y,depth_mean,depth_x,depth_min,width,height
0,711.87683,151.95032,809.37300,308.49020,-0.20,206,1.353822,740,0.924987,97.49617,156.53988
1,390.57785,181.78027,421.47256,200.24520,1.85,199,8.203516,402,7.598885,30.89471,18.46493
2,655.78680,190.24402,698.48170,221.83864,-1.67,215,5.255319,688,4.330514,42.69490,31.59462
3,612.12930,176.14651,729.66540,282.53200,1.55,223,2.592226,714,1.617154,117.53610,106.38549
4,283.69547,184.47151,343.14720,211.07599,1.96,205,5.243353,289,4.655178,59.45173,26.60448


In [7]:
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X_train)
#scalery = StandardScaler().fit(y_train)

In [8]:
X_train= scalerX.transform(X_train)
#y_train_scale = scalery.transform(y_train)

X_valid= scalerX.transform(X_valid)
#y_valid_scale = scalery.transform(y_valid)

X_test= scalerX.transform(X_test)
#y_test_scale = scalery.transform(y_test)

In [9]:
#텐서화
#X_train = xgb.DMatrix(X_train_scale)
#y_train = xgb.DMatrix(y_train.values)

#X_valid = xgb.DMatrix(X_valid_scale)
#y_valid = xgb.DMatrix(y_valid.values)

#X_test = xgb.DMatrix(X_test_scale)
#y_test = xgb.DMatrix(y_test.values)

In [10]:
os.makedirs('./weights', exist_ok=True)

In [11]:
kfol=KFold(n_splits=3,random_state=2022,shuffle=True)
xgb1 = xgb.XGBRegressor( tree_method='gpu_hist')
"""
parameters = {
              'objective':['reg:squarederror'],
              'learning_rate': [0.01,1e-5], 
              'max_depth': [3,5,7,9],
              'min_child_weight': [1,3,4,5],
              'subsample': [0.6,0.7,0.8,1],
              'colsample_bytree':[i/10.0 for i in range(6,10)],
              'n_estimators': [100,200,500,750,1000]
              #'gamma':[i/10.0 for i in range(0,5)],
              #'reg_lambda':[1,0.9,0.8,0.7,0],
              #'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
              }
"""
"""
parameters={
    'n_estimators': [100,200,500,750,1000],
    'max_depth': [3,5,7,9],
    'min_child_weight': [1,3,5],
    'gamma':[i/10.0 for i in range(0,5)],
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
    'learning_rate': [0.01, 0.02, 0.05, 0.1]
}"""

parameters= {'colsample_bytree': [0.9], 
              'learning_rate': [0.01], 
              'max_depth': [9], 
              'min_child_weight': [3], 
              'n_estimators': [1000], 
              'objective': ['reg:squarederror'], 
              'subsample': [0.7],
              'gamma':[i/10.0 for i in range(0,5)],
              'reg_lambda':[1,0.9,0.8,0.7,0],
              'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1, 1],
              }

In [12]:
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = kfol,
                        n_jobs = 3,
                        verbose=0
                        )

In [13]:
xgb_grid.fit(X_train,y_train,early_stopping_rounds=20,eval_metric='rmse',eval_set=[(X_valid,y_valid)],verbose=0)


print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

0.9648946044529213
{'colsample_bytree': 0.9, 'gamma': 0.3, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 1000, 'objective': 'reg:squarederror', 'reg_alpha': 1, 'reg_lambda': 0.9, 'subsample': 0.7}


In [15]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

### 5/5 
bestparm={'colsample_bytree': 0.9,
          'gamma': 0.3,
          'learning_rate': 0.01,
          'max_depth': 9,
          'min_child_weight': 3,
          'n_estimators': 1000,
          'objective': 'reg:squarederror',
          'reg_alpha': 1,
          'reg_lambda': 0.9,
          'subsample': 0.7}

0.9648946044529213
{'colsample_bytree': 0.9, 'gamma': 0.3, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 3, 'n_estimators': 1000, 'objective': 'reg:squarederror', 'reg_alpha': 1, 'reg_lambda': 0.9, 'subsample': 0.7}


In [46]:
"""
resently parameters

best_params= {'colsample_bytree': 0.9, 
              'learning_rate': 0.01, 
              'max_depth': 9, 
              'min_child_weight': 3, 
              'n_estimators': 1000, 
              'objective': 'reg:squarederror', 
              'subsample': 0.7}
"""

In [16]:
d_train=xgb.DMatrix(data=X_train,label=y_train)
d_valid=xgb.DMatrix(data=X_valid,label=y_valid)
wlist=[(d_train,'train'),(d_valid,'eval')]
#best_params=xgb_grid.best_params_

In [17]:
model = xgb.train(params=bestparm,
                  dtrain=d_train,
                 # eval_metric='mae',
                  evals=wlist,
                  num_boost_round=5000
                  #tree_method='gpu_hist',
                )

[19:38:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:31.29034	eval-rmse:31.44952
[1]	train-rmse:30.98396	eval-rmse:31.14216
[2]	train-rmse:30.68044	eval-rmse:30.83746
[3]	train-rmse:30.38029	eval-rmse:30.53630
[4]	train-rmse:30.08305	eval-rmse:30.23790
[5]	train-rmse:29.78878	eval-rmse:29.94292
[6]	train-rmse:29.49748	eval-rmse:29.65059
[7]	train-rmse:29.20923	eval-rmse:29.36110
[8]	train-rmse:28.92333	eval-rmse:29.07410
[9]	train-rmse:28.64297	eval-rmse:28.79209
[10]	train-rmse:28.36309	eval-rmse:28.51176
[11]	train-rmse:28.08599	eval-rmse:28.23325
[12]	train-rmse:27.81185	eval-rmse:27.95783
[13]	train-rmse:2

[178]	train-rmse:6.12885	eval-rmse:6.50235
[179]	train-rmse:6.08076	eval-rmse:6.45817
[180]	train-rmse:6.03342	eval-rmse:6.41526
[181]	train-rmse:5.98662	eval-rmse:6.37293
[182]	train-rmse:5.94042	eval-rmse:6.33165
[183]	train-rmse:5.89495	eval-rmse:6.29014
[184]	train-rmse:5.84986	eval-rmse:6.24895
[185]	train-rmse:5.80565	eval-rmse:6.20938
[186]	train-rmse:5.76178	eval-rmse:6.17024
[187]	train-rmse:5.71860	eval-rmse:6.13158
[188]	train-rmse:5.67570	eval-rmse:6.09270
[189]	train-rmse:5.63352	eval-rmse:6.05511
[190]	train-rmse:5.59129	eval-rmse:6.01771
[191]	train-rmse:5.54985	eval-rmse:5.98180
[192]	train-rmse:5.50888	eval-rmse:5.94505
[193]	train-rmse:5.46851	eval-rmse:5.90919
[194]	train-rmse:5.42832	eval-rmse:5.87387
[195]	train-rmse:5.38853	eval-rmse:5.83893
[196]	train-rmse:5.34982	eval-rmse:5.80503
[197]	train-rmse:5.31091	eval-rmse:5.77061
[198]	train-rmse:5.27271	eval-rmse:5.73735
[199]	train-rmse:5.23494	eval-rmse:5.70373
[200]	train-rmse:5.19787	eval-rmse:5.67089
[201]	train

[369]	train-rmse:2.53713	eval-rmse:3.70457
[370]	train-rmse:2.53256	eval-rmse:3.70227
[371]	train-rmse:2.52757	eval-rmse:3.69988
[372]	train-rmse:2.52320	eval-rmse:3.69764
[373]	train-rmse:2.51858	eval-rmse:3.69553
[374]	train-rmse:2.51408	eval-rmse:3.69390
[375]	train-rmse:2.50992	eval-rmse:3.69200
[376]	train-rmse:2.50570	eval-rmse:3.69030
[377]	train-rmse:2.50069	eval-rmse:3.68763
[378]	train-rmse:2.49672	eval-rmse:3.68605
[379]	train-rmse:2.49310	eval-rmse:3.68472
[380]	train-rmse:2.48909	eval-rmse:3.68320
[381]	train-rmse:2.48488	eval-rmse:3.68189
[382]	train-rmse:2.47976	eval-rmse:3.67934
[383]	train-rmse:2.47585	eval-rmse:3.67770
[384]	train-rmse:2.47180	eval-rmse:3.67585
[385]	train-rmse:2.46795	eval-rmse:3.67399
[386]	train-rmse:2.46411	eval-rmse:3.67246
[387]	train-rmse:2.45933	eval-rmse:3.67044
[388]	train-rmse:2.45540	eval-rmse:3.66896
[389]	train-rmse:2.45094	eval-rmse:3.66737
[390]	train-rmse:2.44784	eval-rmse:3.66605
[391]	train-rmse:2.44377	eval-rmse:3.66382
[392]	train

[560]	train-rmse:2.08454	eval-rmse:3.56875
[561]	train-rmse:2.08343	eval-rmse:3.56865
[562]	train-rmse:2.08204	eval-rmse:3.56822
[563]	train-rmse:2.08009	eval-rmse:3.56818
[564]	train-rmse:2.07910	eval-rmse:3.56834
[565]	train-rmse:2.07778	eval-rmse:3.56833
[566]	train-rmse:2.07706	eval-rmse:3.56853
[567]	train-rmse:2.07619	eval-rmse:3.56830
[568]	train-rmse:2.07538	eval-rmse:3.56832
[569]	train-rmse:2.07402	eval-rmse:3.56808
[570]	train-rmse:2.07325	eval-rmse:3.56790
[571]	train-rmse:2.07220	eval-rmse:3.56802
[572]	train-rmse:2.07174	eval-rmse:3.56826
[573]	train-rmse:2.07032	eval-rmse:3.56788
[574]	train-rmse:2.06869	eval-rmse:3.56764
[575]	train-rmse:2.06766	eval-rmse:3.56741
[576]	train-rmse:2.06568	eval-rmse:3.56699
[577]	train-rmse:2.06446	eval-rmse:3.56677
[578]	train-rmse:2.06350	eval-rmse:3.56637
[579]	train-rmse:2.06217	eval-rmse:3.56628
[580]	train-rmse:2.06103	eval-rmse:3.56579
[581]	train-rmse:2.05973	eval-rmse:3.56546
[582]	train-rmse:2.05858	eval-rmse:3.56529
[583]	train

[751]	train-rmse:1.88909	eval-rmse:3.54901
[752]	train-rmse:1.88825	eval-rmse:3.54909
[753]	train-rmse:1.88762	eval-rmse:3.54914
[754]	train-rmse:1.88695	eval-rmse:3.54921
[755]	train-rmse:1.88586	eval-rmse:3.54917
[756]	train-rmse:1.88458	eval-rmse:3.54886
[757]	train-rmse:1.88377	eval-rmse:3.54868
[758]	train-rmse:1.88282	eval-rmse:3.54874
[759]	train-rmse:1.88200	eval-rmse:3.54860
[760]	train-rmse:1.88158	eval-rmse:3.54863
[761]	train-rmse:1.88110	eval-rmse:3.54884
[762]	train-rmse:1.88081	eval-rmse:3.54874
[763]	train-rmse:1.87996	eval-rmse:3.54851
[764]	train-rmse:1.87920	eval-rmse:3.54845
[765]	train-rmse:1.87857	eval-rmse:3.54817
[766]	train-rmse:1.87764	eval-rmse:3.54815
[767]	train-rmse:1.87681	eval-rmse:3.54814
[768]	train-rmse:1.87538	eval-rmse:3.54785
[769]	train-rmse:1.87456	eval-rmse:3.54775
[770]	train-rmse:1.87331	eval-rmse:3.54771
[771]	train-rmse:1.87232	eval-rmse:3.54776
[772]	train-rmse:1.87057	eval-rmse:3.54734
[773]	train-rmse:1.86954	eval-rmse:3.54735
[774]	train

...

[4122]	train-rmse:0.61988	eval-rmse:3.46465
[4123]	train-rmse:0.61967	eval-rmse:3.46466
[4124]	train-rmse:0.61955	eval-rmse:3.46465
[4125]	train-rmse:0.61940	eval-rmse:3.46467
[4126]	train-rmse:0.61919	eval-rmse:3.46463
[4127]	train-rmse:0.61903	eval-rmse:3.46462
[4128]	train-rmse:0.61883	eval-rmse:3.46460
[4129]	train-rmse:0.61874	eval-rmse:3.46462
[4130]	train-rmse:0.61853	eval-rmse:3.46463
[4131]	train-rmse:0.61837	eval-rmse:3.46461
[4132]	train-rmse:0.61819	eval-rmse:3.46460
[4133]	train-rmse:0.61814	eval-rmse:3.46458
[4134]	train-rmse:0.61804	eval-rmse:3.46459
[4135]	train-rmse:0.61791	eval-rmse:3.46456
[4136]	train-rmse:0.61765	eval-rmse:3.46456
[4137]	train-rmse:0.61746	eval-rmse:3.46454
[4138]	train-rmse:0.61727	eval-rmse:3.46453
[4139]	train-rmse:0.61715	eval-rmse:3.46457
[4140]	train-rmse:0.61699	eval-rmse:3.46456
[4141]	train-rmse:0.61684	eval-rmse:3.46458
[4142]	train-rmse:0.61668	eval-rmse:3.46454
[4143]	train-rmse:0.61646	eval-rmse:3.46451
[4144]	train-rmse:0.61631	eval-r

[4309]	train-rmse:0.59041	eval-rmse:3.46389
[4310]	train-rmse:0.59029	eval-rmse:3.46395
[4311]	train-rmse:0.59018	eval-rmse:3.46396
[4312]	train-rmse:0.59000	eval-rmse:3.46394
[4313]	train-rmse:0.58987	eval-rmse:3.46396
[4314]	train-rmse:0.58968	eval-rmse:3.46395
[4315]	train-rmse:0.58951	eval-rmse:3.46395
[4316]	train-rmse:0.58943	eval-rmse:3.46396
[4317]	train-rmse:0.58924	eval-rmse:3.46395
[4318]	train-rmse:0.58921	eval-rmse:3.46395
[4319]	train-rmse:0.58905	eval-rmse:3.46398
[4320]	train-rmse:0.58896	eval-rmse:3.46399
[4321]	train-rmse:0.58883	eval-rmse:3.46398
[4322]	train-rmse:0.58872	eval-rmse:3.46398
[4323]	train-rmse:0.58864	eval-rmse:3.46398
[4324]	train-rmse:0.58842	eval-rmse:3.46397
[4325]	train-rmse:0.58830	eval-rmse:3.46399
[4326]	train-rmse:0.58815	eval-rmse:3.46397
[4327]	train-rmse:0.58809	eval-rmse:3.46397
[4328]	train-rmse:0.58800	eval-rmse:3.46397
[4329]	train-rmse:0.58783	eval-rmse:3.46395
[4330]	train-rmse:0.58769	eval-rmse:3.46392
[4331]	train-rmse:0.58749	eval-r

[4496]	train-rmse:0.56369	eval-rmse:3.46324
[4497]	train-rmse:0.56352	eval-rmse:3.46324
[4498]	train-rmse:0.56342	eval-rmse:3.46324
[4499]	train-rmse:0.56323	eval-rmse:3.46326
[4500]	train-rmse:0.56315	eval-rmse:3.46325
[4501]	train-rmse:0.56300	eval-rmse:3.46321
[4502]	train-rmse:0.56294	eval-rmse:3.46322
[4503]	train-rmse:0.56270	eval-rmse:3.46320
[4504]	train-rmse:0.56253	eval-rmse:3.46321
[4505]	train-rmse:0.56249	eval-rmse:3.46321
[4506]	train-rmse:0.56234	eval-rmse:3.46320
[4507]	train-rmse:0.56218	eval-rmse:3.46321
[4508]	train-rmse:0.56202	eval-rmse:3.46319
[4509]	train-rmse:0.56195	eval-rmse:3.46319
[4510]	train-rmse:0.56189	eval-rmse:3.46317
[4511]	train-rmse:0.56180	eval-rmse:3.46317
[4512]	train-rmse:0.56175	eval-rmse:3.46319
[4513]	train-rmse:0.56166	eval-rmse:3.46319
[4514]	train-rmse:0.56154	eval-rmse:3.46321
[4515]	train-rmse:0.56142	eval-rmse:3.46320
[4516]	train-rmse:0.56119	eval-rmse:3.46317
[4517]	train-rmse:0.56100	eval-rmse:3.46313
[4518]	train-rmse:0.56084	eval-r

[4682]	train-rmse:0.53943	eval-rmse:3.46263
[4683]	train-rmse:0.53936	eval-rmse:3.46266
[4684]	train-rmse:0.53922	eval-rmse:3.46266
[4685]	train-rmse:0.53910	eval-rmse:3.46270
[4686]	train-rmse:0.53894	eval-rmse:3.46270
[4687]	train-rmse:0.53882	eval-rmse:3.46270
[4688]	train-rmse:0.53864	eval-rmse:3.46271
[4689]	train-rmse:0.53845	eval-rmse:3.46269
[4690]	train-rmse:0.53834	eval-rmse:3.46271
[4691]	train-rmse:0.53819	eval-rmse:3.46270
[4692]	train-rmse:0.53810	eval-rmse:3.46272
[4693]	train-rmse:0.53791	eval-rmse:3.46272
[4694]	train-rmse:0.53781	eval-rmse:3.46274
[4695]	train-rmse:0.53762	eval-rmse:3.46272
[4696]	train-rmse:0.53753	eval-rmse:3.46275
[4697]	train-rmse:0.53729	eval-rmse:3.46274
[4698]	train-rmse:0.53720	eval-rmse:3.46273
[4699]	train-rmse:0.53707	eval-rmse:3.46273
[4700]	train-rmse:0.53698	eval-rmse:3.46272
[4701]	train-rmse:0.53686	eval-rmse:3.46271
[4702]	train-rmse:0.53672	eval-rmse:3.46273
[4703]	train-rmse:0.53658	eval-rmse:3.46274
[4704]	train-rmse:0.53642	eval-r

[4869]	train-rmse:0.51544	eval-rmse:3.46251
[4870]	train-rmse:0.51529	eval-rmse:3.46248
[4871]	train-rmse:0.51515	eval-rmse:3.46250
[4872]	train-rmse:0.51502	eval-rmse:3.46251
[4873]	train-rmse:0.51491	eval-rmse:3.46251
[4874]	train-rmse:0.51477	eval-rmse:3.46253
[4875]	train-rmse:0.51471	eval-rmse:3.46254
[4876]	train-rmse:0.51458	eval-rmse:3.46253
[4877]	train-rmse:0.51441	eval-rmse:3.46251
[4878]	train-rmse:0.51415	eval-rmse:3.46249
[4879]	train-rmse:0.51405	eval-rmse:3.46248
[4880]	train-rmse:0.51384	eval-rmse:3.46247
[4881]	train-rmse:0.51368	eval-rmse:3.46247
[4882]	train-rmse:0.51357	eval-rmse:3.46245
[4883]	train-rmse:0.51343	eval-rmse:3.46244
[4884]	train-rmse:0.51330	eval-rmse:3.46241
[4885]	train-rmse:0.51313	eval-rmse:3.46240
[4886]	train-rmse:0.51300	eval-rmse:3.46237
[4887]	train-rmse:0.51283	eval-rmse:3.46238
[4888]	train-rmse:0.51275	eval-rmse:3.46238
[4889]	train-rmse:0.51252	eval-rmse:3.46240
[4890]	train-rmse:0.51236	eval-rmse:3.46239
[4891]	train-rmse:0.51226	eval-r

In [18]:
d_test=xgb.DMatrix(data=X_test,label=y_test)

In [19]:
preds= model.predict(d_test)

In [22]:
def testing(pred,y):
    sums1=[]
    sums2=[]
    for i in range(len(preds)):
        sqr=(float(pred[i])-float(y.loc[i]))**2
        abs1=abs(float(pred[i])-float(y.loc[i]))
        sums1.append(sqr)
        sums2.append(abs1)
        
        
    rmse=(sum(sums1)/len(preds))**0.5
    mae= sum(sums2)/len(preds)
    return rmse,mae



        

In [24]:
print('rmse =', testing(preds,y_test)[0])
print('mae =', testing(preds,y_test)[1])

rmse = 3.3840703064722164
mae = 1.6726702457998583
